In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance
# !git pull

%cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies

In [ ]:
import os
import sys
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../wae/")

import torch
import numpy as np
import matplotlib 
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D  
import pandas as pd
import itertools

from conc_environment_factory import ConcEnvironmentFactory
from conc_build_parameter import ConcBuildParameter
from conc_build_parameter_factory import ConcBuildParameterFactory
from conc_agent_factory import ConcAgentFactory
from wae_trainer_factory import WaeTrainerFactory

from wae_batch_data_agent import WaeBatchDataAgent
from wae_batch_data_environment import WaeBatchDataEnvironment

from builder import Builder
from store import Store
from mylogger import MyLogger

from loader import Loader

In [ ]:
dbPath = "training_log.sqlite"

In [ ]:
target_casestudy = "cs03b"

In [ ]:
def evaluate_err(buildParameter, agent):
    environment = environmentFactory.create(buildParameter)
    trainer = trainerFactory.create(buildParameter, agent, environment)

    dataBatchEnv = environment.getTestData()
    dataBatchAg = agent(dataBatchEnv)

    _err_observable = torch.mean(torch.abs(dataBatchAg._XHat - dataBatchEnv._X))
    _err_latent, _ = trainer.measure_distance(dataBatchAg._Xi, dataBatchAg._XiHat)

    err_observable = _err_observable.data.numpy()
    err_latent = _err_latent.data.numpy()    
    return err_observable, err_latent

In [ ]:
def approximate_observable_image(th, ph, R, r):
    xi1 = (R + r * np.cos(th)) * np.cos(ph)
    xi2 = (R + r * np.cos(th)) * np.sin(ph)
    xi3 = r * np.sin(th)

    X1 = np.stack((xi1, xi2, xi3), axis=-1) # (..., 3)
    X2 = np.stack((R-xi2, xi3, -xi1), axis=-1) # (..., 3)

    return X1, X2 # (..., 3)


def approximate_latent_image(th, ph, R, r):
    xi1 = (R + r * np.cos(th)) * np.cos(ph)
    xi2 = (R + r * np.cos(th)) * np.sin(ph)
    xi3 = r * np.sin(th)
    
    Xi = np.stack((xi1, xi2, xi3), axis=-1) # (..., nXi = 3)
    
    Xi1 = Xi.copy()
    Xi1[...,2] += R/2
    Xi2 = Xi.copy()
    Xi2[...,2] -= R/2
    
    return Xi1, Xi2 # (..., 3)

In [ ]:
def plot_encoder_projection_image(agent, environment, azim):

    nPh = 2**6
    nTh = 2**6
    ph = np.linspace(0, 1, nPh) * np.pi * 2
    th = np.linspace(0, 1, nTh) * np.pi * 2
    Ph, Th = np.meshgrid(ph, th) # (nTh, nPh)
    XYZs = approximate_observable_image(Ph, Th, environment.R, environment.r) # (nTh, nPh, 3)
    Xis = approximate_latent_image(Ph, Th, environment.R, environment.r) # (nTh, nPh, 3)

    _XYZs = [
        torch.from_numpy(XYZ.astype(np.float32).reshape(-1,3)) # (nTh * nPh, 3)
        for XYZ in XYZs]

    _Zs = [ torch.zeros(nTh * nPh, environment.nZ) # (*, nZ = 2) 
           for k1 in range(2)]

    XiHats = [None, None]
    for k1 in range(2):
        dataBatchEnv = WaeBatchDataEnvironment(_XYZs[k1], _Zs[k1])
        dataBatchAgent = agent(dataBatchEnv)
        XiHats[k1] = dataBatchAgent._XiHat.data.numpy().reshape(nTh, nPh, -1) # (nTheta, nR, 3)

    colors = ["red", "blue"]
    fig = plt.gcf()
    #
    ax = fig.add_subplot(1,2,1, projection = "3d")
    for k1, XYZ in enumerate(XYZs):        
        #     ax.plot_surface(XYZ[...,0], XYZ[...,1], XYZ[...,2], color = colors[k1], linewidth = 0, antialiased=True)
        ax.plot_wireframe(XYZ[...,0], XYZ[...,1], XYZ[...,2], color = colors[k1], linewidth = .5, alpha=.5, antialiased=True)
    ax.set_title('Input image \non the observable variables space')
    ax.axis('off')
    ax.view_init(30, azim)
    #
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-1.10,1.10)
    #
    ax = fig.add_subplot(1,2,2, projection = "3d")
    for k1, (Xi, XiHat) in enumerate(zip(Xis, XiHats)): 
        ax.plot_wireframe(Xi[...,0], Xi[...,1], Xi[...,2], color='lightgrey', linewidth=.5, alpha = .5, antialiased=True)
        #     ax.plot_surface(XiHat[...,0], XiHat[...,1], XiHat[...,2], color=colors[k1], linewidth = 0, antialiased=True)
        ax.plot_wireframe(XiHat[...,0], XiHat[...,1], XiHat[...,2], color=colors[k1], linewidth = .5, antialiased=True)
    ax.set_title('Ouput image \non the latent variables space')
    ax.axis('off')
    ax.view_init(30, azim)
    #
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-1.10,1.10)
    fig.tight_layout()

In [ ]:
def plot_decoder_projection_image(agent, environment, azim):

    nPh = 2**6
    nTh = 2**6
    ph = np.linspace(0, 1, nPh) * np.pi * 2
    th = np.linspace(0, 1, nTh) * np.pi * 2
    Ph, Th = np.meshgrid(ph, th) # (nTh, nPh)
    XYZs = approximate_observable_image(Ph, Th, environment.R, environment.r) # (nTh, nPh, 3)
    Xis = approximate_latent_image(Ph, Th, environment.R, environment.r) # (nTh, nPh, 3)

    XYZHats = [None, None]
    for k1 in range(2):
        _Xi = torch.from_numpy(Xis[k1].astype(np.float32).reshape(nTh*nPh, 3)) # (nTh * nPh, 3)
        _XYZHat = agent.dec(_Xi) # (nTh * nPh, 3)
        XYZHats[k1] = _XYZHat.data.numpy().reshape(nTh, nPh, 3) # (nTh, nPh, 3)

    colors = ["red", "blue"]
    fig = plt.gcf()
    #
    ax = fig.add_subplot(1,2,1, projection = "3d")
    for k1, Xi in enumerate(Xis):        
        ax.plot_wireframe(Xi[...,0], Xi[...,1], Xi[...,2], color=colors[k1], linewidth=.5, alpha = .5, antialiased=True)

    ax.set_title('Input image \non the latent variables space')
    ax.axis('off')
    ax.view_init(30, azim)
    #
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-1.10,1.10)
    #
    ax = fig.add_subplot(1,2,2, projection = "3d")
    for k1, (XYZ, XYZHat) in enumerate(zip(XYZs, XYZHats)): 
        ax.plot_wireframe(XYZ[...,0], XYZ[...,1], XYZ[...,2], color = 'lightgray', linewidth = .5, alpha=.5, antialiased=True)
        ax.plot_wireframe(XYZHat[...,0], XYZHat[...,1], XYZHat[...,2], color=colors[k1], linewidth = .5, antialiased=True)
    ax.set_title('Ouput image \non the observable variables space')
    ax.axis('off')
    ax.view_init(30, azim)
    #
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-1.10,1.10)
    fig.tight_layout()

## S400: Load trained agents to analyze them

### SS410: initialize a loader of trained agents

In [ ]:
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()
trainerFactory =  WaeTrainerFactory()

store = Store(dbPath)

buildParameterFactory = ConcBuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

### SS420: evaluate trained agents:

evaluation error is here:
* the representitive errors of observable variables
* and the discrepancy between latent referenced distribution and the one projected by trained encoder

In [ ]:
tbl = {
    "criteria": []
    , "score": []
    , "epoch": []
      }
for agent, buildParameter, epoch in loader.load(target_casestudy + "%"):
    representative_error, latent_distribution_discrepancy = evaluate_err(buildParameter, agent)
    
    for score, criteria in [
        (representative_error, "Representative Error")
        , (latent_distribution_discrepancy, "Latent Distributions Discrepancy")]:

        for key in buildParameter.__dict__:
            if not key in tbl:
                tbl[key] = []
            tbl[key].append(buildParameter.__dict__[key])
    
        tbl["epoch"].append(epoch)
        tbl["criteria"].append(criteria)
        tbl["score"].append(score)

tbl = pd.DataFrame(tbl)
tbl.to_csv(target_casestudy +  "_score.csv")

### SS430: Print encoder's images

In [ ]:
for agent, buildParameter, epoch in loader.load(target_casestudy + "%", buildParameterKey=None):
    
    if epoch == buildParameter.nEpoch:
        for azim in np.arange(0, 360, 30):
            environment = environmentFactory.create(buildParameter)
            fig = plt.figure(figsize=[12, 6])
            plot_encoder_projection_image(agent, environment, azim)
            fig.savefig("./img/%s_encoder_projection_%s_azim=%03d.png" % (target_casestudy, buildParameter.key, azim))
            plt.close(fig)
        break

### SS440: Print decoder's images

In [ ]:
for agent, buildParameter, epoch in loader.load(target_casestudy + "%", buildParameterKey=None):
    
    if epoch == buildParameter.nEpoch:
        for azim in np.arange(0, 360, 30):
            environment = environmentFactory.create(buildParameter)
            fig = plt.figure(figsize=[12, 6])
            plot_decoder_projection_image(agent, environment, azim)
            fig.savefig("./img/%s_decoder_projection_%s_azim=%03d.png" % (target_casestudy, buildParameter.key, azim))
            plt.close(fig)
        break